In [ ]:
#default_exp searchKevin

# Search Function by Kevin

search with a new version with the following goals

Use Dismax / Allow sort by: / price / best match (default)/ Allow filter by/ category/ min*max price/ multiple terms/ Faster render


In [ ]:
#export
import boto3
from dataclasses import dataclass, field
from dataclasses_json import dataclass_json
from typing import List, Any, Optional
from functools import cached_property
from awsSchema.apigateway import Response, Event
import pandas as pd
from spellchecker import SpellChecker
from operator import itemgetter
import phonetics
from difflib import SequenceMatcher
import json, asyncio
import threading
from src.getEnv import BRANCH
from nicHelper.exception import errorString

In [ ]:
#export
@dataclass_json
@dataclass
class QueryInput:
    query : str
    allBrand: List[str] = None
    allFilter: Optional[List[str]] = None

@dataclass_json
@dataclass
class Item:
    cprcodes: str
    thaiName: str
    engName: str
    price: str
    brand: str
    filters: Optional[List[str]] = None

@dataclass_json
@dataclass
class Input:
    '''
    input class, search function should take this as input
    '''
    queryStringParameters: QueryInput

In [ ]:
#export
@dataclass_json
@dataclass
class Body:
    '''output from your custom search functions'''
    searchResult: List[Item]
    queryInput: QueryInput
    suggestions: List[str] = field(default_factory=list)
    
@dataclass_json
@dataclass
class Output:
    '''
    output class, search function should return this as output
    body should be json payload of class Body
    '''
    body: str #json payload of class Body
    statusCode: int = 200
    headers: dict = field(default_factory=lambda: 
        { 'Access-Control-Allow-Headers': '*', 
            'Access-Control-Allow-Origin': '*', 
            'Access-Control-Allow-Methods': '*'})

In [ ]:
#export
with open('src/soundlist.txt','r') as f:
    common={i.split(' ')[0]:i.split(' ')[1][:-1] for i in f if not(i.split(' ')[1][:-2]=='')}

In [ ]:
#export
@dataclass_json
@dataclass
class Field:
    cprcode:List[str]
    pr_engname:List[str]
    pr_name:List[str]
    hema_brand_en:List[str]
    ba_nprice:List[str]
    pr_filter_en:Optional[List[str]]=None
    @cached_property
    def parsedCprcode(self): return self.cprcode[0]
    @cached_property
    def engName(self): return self.pr_engname[0]
    @cached_property
    def filter(self): return self.pr_filter_en[0]
    @cached_property
    def price(self): return self.ba_nprice[0]
    @cached_property
    def brand(self): return self.hema_brand_en[0]
    @cached_property
    def name(self): return self.pr_name[0]
@dataclass_json
@dataclass
class Result:
    id:str
    fields:Field
@dataclass_json
@dataclass
class Hits:
    hit:List[Result]

@dataclass_json
@dataclass
class SearchResponse:
    hits: Hits
    @cached_property
    def result(self)->List[Field]: return [result.fields for result in self.hits.hit]
    @cached_property
    def resultDict(self)->List[dict]:
        return [{'cprcode':r.parsedCprcode, 'name':r.name} for r in self.result]
    @cached_property
    def resultDf(self)->pd.DataFrame:
        return pd.DataFrame(self.resultDict)
    @cached_property
    def nameSortedDf(self)->pd.DataFrame:
        return self.resultDf.sort_values(by='name')

 

In [ ]:
#export
class Search:
    url = 'https://search-villa-cloudsearch-2-4izacsoytzqf6kztcyjhssy2ti.ap-southeast-1.cloudsearch.amazonaws.com'
    client = boto3.client('cloudsearchdomain', endpoint_url = url)
    direction = ['ba_nprice asc','ba_nprice desc']

    def __init__(self):
        self.suggestions=0
        self.phonetic=0

    def find(self, query,sort)->SearchResponse:
        response = self.client.search(
            query = query,
            queryParser = 'dismax',
            size = 100,
            sort= self.direction[sort],
            returnFields = 'cprcode,pr_engname,pr_filter_en,ba_nprice,pr_name,hema_brand_en'
            )
        return response
    
    def leoPhonetic(self,query):
        words=query.split()
        for word in range(len(words)):
            itemsound=phonetics.soundex(words[word])
            score=0
            for i in common:
                newScore=SequenceMatcher(None, common[i], itemsound).ratio()
                if newScore > score:
                    item=i
                    score=newScore
            words[word]=item
        print(words)
        item=' '.join(words)
        self.phonetic=item


    def suggestion(self,query)->List[dict]:
        candidates = SpellChecker().candidates(query)
        result = []
        keyword = []
        found=[]
        suggestion=[]
        print(candidates)
        if candidates!=None:
            for candidate in candidates:
                print(candidate)
                response = self.find(candidate,0)
                if response['hits']['found'] != 0:
                    result.append(response)
                    keyword.append(candidate)
                    found.append(response['hits']['found'])
        response = [ {"keyword":val[0], "result":val[1], "found":val[2]} for val in zip(keyword, result, found)]
        rank = sorted(response, key=itemgetter('found'),reverse=True)
        for item in rank:
            suggestion.append(item['keyword'])
        self.suggestions=suggestion

        
    
    def getAllFilter(self)->List[str]:
        filters=[]
        for i in [filterlist.fields.pr_filter_en for filterlist in self.hits.hit if filterlist.fields.pr_filter_en != None]:
            for j in i[0].split(','):
            
                if j not in filters:
                    filters.append(j)
        return filters

    @staticmethod
    def recordSearch(searchTerm: str, cprcodes: List[str]):
        try:
            lambda_client = boto3.client('lambda')
            lambda_client.invoke_async(
                FunctionName=f'record-search-{BRANCH}',
                InvokeArgs=json.dumps({'searchTerm': searchTerm, 'cprcodes': cprcodes})
            )
        except Exception as e:
            print("can't invoke record search function: ", e, errorString())
            # sentry here

    def search(self,query, returnDataclass:bool=False):

        usersInput = self.find(query,0)
        print(usersInput['hits']['found'])
        suggestion = []
        if usersInput['hits']['found'] <= 10: # Kevin, this needs a change. If there are 8 results, they need to be shown first

            # t1 = threading.Thread(target=self.leoPhonetic(query), name='t1')
            # t2 = threading.Thread(target=self.suggestion(query), name='t2')
         

            # t1.start()
            # t2.start()
            # t1.join()
            # t2.join()
            self.leoPhonetic(query)
            # suggestion = self.suggestions
            phonetic = self.phonetic

            # if phonetic in suggestion:
            #     suggestion.remove(phonetic)
            response = self.find(phonetic,0)

        else:
            response=usersInput


        jsonClassResponse = SearchResponse.from_dict(response)
        # print("JSONCLASSRESPONSE :: ", jsonClassResponse)
        tempOutputDict = {}
        cprcodes=[x.fields.cprcode[0] for x in jsonClassResponse.hits.hit]
        price=[x.fields.ba_nprice[0] for x in jsonClassResponse.hits.hit]
        thaiName=[x.fields.pr_name[0] for x in jsonClassResponse.hits.hit]
        engName=[x.fields.pr_engname[0] for x in jsonClassResponse.hits.hit]
        brand=[x.fields.hema_brand_en[0] for x in jsonClassResponse.hits.hit]
        filters = []
        for x in jsonClassResponse.hits.hit:
            if x.fields.pr_filter_en!= None:
                filters.append(x.fields.pr_filter_en[0] )
            else:
                filters.append(None)

        for i,x in enumerate(filters):
            if x!=None:
                z=[]
                for y in x.split(','):
                    z= z + y.split(' ')
    
                z=list(filter(lambda a: a != 'and'and a!='', z))
                
                filters[i]=z
        allFilter=[]
        for x in filters:
            if x!=None:
                for y in x:
                    if y not in allFilter:
                        allFilter.append(y)
        
        allBrand = []
        for x in brand:
            if x not in allBrand:
                allBrand.append(x)
        # print(allBrand)
        
        tempOutputDict['searchResult'] = [ {"cprcodes":val[0], "price":val[1], "thaiName":val[2],"engName":val[3],"brand":val[4],"filters":val[5]} for val in zip(cprcodes, price, thaiName,engName,brand,filters)]
        # print(tempOutputDict['searchResult'])
        # tempOutputDict['cprcodes'] = [x.fields.cprcode[0] for x in jsonClassResponse.hits.hit]
        print(filters)
        tempOutputDict['queryInput'] = {'query':query,'allFilter':allFilter,'allBrand':allBrand}
        tempOutputDict['suggestions'] = suggestion
        
        # print("TEMP OUTPUT DICT :: ", tempOutputDict)
        outputBody = Body.from_dict(tempOutputDict)
        if returnDataclass:
            return outputBody
        # print("OUTPUT BODY :: ", outputBody)

        output = Output(body=json.dumps(outputBody.to_dict()))
        # print("OUTPUT :: ", output)
        
        return output.to_dict()
       

In [ ]:
#export
def searchKevin(event, *args):
    params = Input.from_dict(event)
    # print("PARAMS :: ", params)
    searchClient = Search() 
    result = searchClient.search(params.queryStringParameters.query)
    Search.recordSearch(params.queryStringParameters.query, [x['cprcodes'] for x in json.loads(result['body'])['searchResult']])
    return Response.returnSuccess(result)
    

In [ ]:
import time
start_time = time.time()
res = searchKevin(Event.getInput(queryStringParameters={'query':'chicken'}))
print("--- %s seconds ---" % (time.time() - start_time))
# pd.DataFrame(res)
# res.filterlist
# for i in res.hits.hit:
#     print(i)
# print(len(res.hits.hit))
# print(len([i for i in res if i.hits.hit.fields.pr_filter_en]))
# res


592
[['Ready', 'Meal', 'Fresh'], ['Fresh', 'Cooked', 'Meats'], None, None, None, None, None, None, None, None, None, None, None, None, ['Fresh', 'Fresh', 'Chicken', 'Poultry'], None, None, None, ['Condiments', 'Dressings', 'Grocery'], None, ['Fresh', 'Ready', 'Meal'], ['Spices', 'Paste', 'Grocery'], ['Grocery', 'Cooking', 'Sauces', 'Gravy'], None, ['Cooked', 'Meats', 'Fresh'], None, None, ['Cans', 'Packets', 'Grocery'], ['Cooked', 'Meats', 'Fresh'], ['Frozen', 'NULL', 'Pastries', 'Bread'], ['Cans', 'Packets', 'Grocery'], ['Asian', 'Meal', 'Frozen'], ['Cooked', 'Meats', 'Fresh'], None, None, ['Fresh', 'Delicatessen', 'Dips'], ['Delicatessen', 'Dips', 'Cooked', 'Meats', 'Fresh'], None, ['Fresh', 'Ready', 'Meal'], ['Fresh', 'Ready', 'Meal'], None, ['Cans', 'Packets', 'Grocery'], None, None, ['Condiments', 'Dressings', 'Grocery'], None, None, ['Fresh', 'Ready', 'Meal'], ['Rice', 'Pasta', 'Beans', 'Grocery'], None, ['Cooking', 'Sauces', 'Gravy', 'Grocery'], ['Grocery', 'Condiments', 'Dressi

In [ ]:
url = 'https://search-villa-cloudsearch-2-4izacsoytzqf6kztcyjhssy2ti.ap-southeast-1.cloudsearch.amazonaws.com'
client = boto3.client('cloudsearchdomain', endpoint_url = url)
direction = ['ba_nprice asc','ba_nprice desc']
query = 'chicken'
response = client.search(
            query = query,
            queryParser = 'dismax',
            size = 100,
            sort= direction[0]
            )
response

{'ResponseMetadata': {'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json; charset=UTF-8',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'status': {'timems': 3, 'rid': '6Zaardgw+94bCtQnQw=='},
 'hits': {'found': 591,
  'start': 0,
  'hit': [{'id': '192947',
    'fields': {'cprcode': ['192947'],
     'iprcode': ['192947'],
     'oprcode': ['192947'],
     'pr_abb': ['FRIED EGG'],
     'pr_active': ['True'],
     'pr_engname': ['BETAGRO FRIED EGG'],
     'pr_name': ['BETAGRO ไข่ดาว'],
     'pr_country_en': ['TH'],
     'pr_filter_th': ['Ready Meal,Fresh'],
     'pr_filter_en': ['Ready Meal,Fresh'],
     'online_category_l1_en': ['Delicatessen & Eggs'],
     'online_category_l2_en': ['Deli Meats'],
     'online_category_l3_en': ['Chicken & Poultry'],
     'villa_category_l1_en': ['Fresh'],
     'villa_category_l2_en': ['Delicatessen'],
     'villa_category_l3_en': ['Local'],
     'villa_category_l4_en': ['Duck'],
     'co